# Read Data Sample

In [1]:
import pandas as pd
import numpy as np
import os
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

In [2]:
class dataset:
    kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
    kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")
    
    kdd_train_5labels = pd.read_pickle("dataset/kdd_train_5labels.pkl")
    kdd_test_5labels = pd.read_pickle("dataset/kdd_test_5labels.pkl")
    

In [3]:
dataset.kdd_train_2labels.shape

(125973, 124)

In [4]:
dataset.kdd_test_2labels.shape

(22544, 124)

In [5]:
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

class preprocess:
    
    output_columns_2labels = ['is_Attack','is_Normal']
    
    x_input = dataset.kdd_train_2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.kdd_train_2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.kdd_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.kdd_test_2labels.loc[:,output_columns_2labels]

    ss = pp.StandardScaler()

    x_train = ss.fit_transform(x_input)
    x_test = ss.transform(x_test_input)

    y_train = y_output.values
    y_test = y_test.values

preprocess.x_train.std()

0.99589320646770185

In [6]:
import tensorflow as tf


In [7]:
class network(object):
    
    input_dim = 122
    classes = 2
    hidden_encoder_dim = 122
    hidden_layers = 1
    latent_dim = 10

    hidden_decoder_dim = 122
    lam = 0.001
    
    def __init__(self, classes, hidden_layers, num_of_features):
        self.classes = classes
        self.hidden_layers = hidden_layers
        self.latent_dim = num_of_features
            
    def build_layers(self):
        tf.reset_default_graph()
        #learning_rate = tf.Variable(initial_value=0.001)

        input_dim = self.input_dim
        classes = self.classes
        hidden_encoder_dim = self.hidden_encoder_dim
        hidden_layers = self.hidden_layers
        latent_dim = self.latent_dim
        hidden_decoder_dim = self.hidden_decoder_dim
        lam = self.lam
        
        with tf.variable_scope("Input"):
            self.x = tf.placeholder("float", shape=[None, input_dim])
            self.y_ = tf.placeholder("float", shape=[None, classes])
            self.keep_prob = tf.placeholder("float")
        
        with tf.variable_scope("Layer_Encoder"):

            hidden_encoder = tf.layers.dense(self.x, hidden_encoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
            for h in range(hidden_layers - 1):
                hidden_encoder = tf.layers.dense(hidden_encoder, hidden_encoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
                hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
                
        with tf.variable_scope("Layer_Mean"):
            mu_encoder = tf.layers.dense(hidden_encoder, latent_dim, activation = None, kernel_regularizer=tf.nn.l2_loss)

        with tf.variable_scope("Layer_Variance"):
            logvar_encoder = tf.layers.dense(hidden_encoder, latent_dim, activation = None, kernel_regularizer=tf.nn.l2_loss)

        with tf.variable_scope("Sampling_Distribution"):
            # Sample epsilon
            epsilon = tf.random_normal(tf.shape(logvar_encoder), mean=0, stddev=1, name='epsilon')

            # Sample latent variable
            std_encoder = tf.exp(0.5 * logvar_encoder)
            z = mu_encoder + tf.multiply(std_encoder, epsilon)
            
            #tf.summary.histogram("Sample_Distribution", z)

        with tf.variable_scope("Layer_Decoder"):
            hidden_decoder = tf.layers.dense(z, hidden_decoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            hidden_decoder = tf.nn.dropout(hidden_decoder, self.keep_prob)
            for h in range(hidden_layers - 1):
                hidden_decoder = tf.layers.dense(hidden_decoder, hidden_decoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
                hidden_decoder = tf.nn.dropout(hidden_decoder, self.keep_prob)
                
        with tf.variable_scope("Layer_Reconstruction"):
            x_hat = tf.layers.dense(hidden_decoder, input_dim, activation = None)
            
        with tf.variable_scope("Layer_Dense_Hidden"):
            hidden_output = tf.layers.dense(z,latent_dim, activation=tf.nn.relu)

        with tf.variable_scope("Layer_Dense_Softmax"):
            self.y = tf.layers.dense(z, classes, activation=tf.nn.softmax)

        with tf.variable_scope("Loss"):
            
            BCE = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=x_hat, labels=self.x), reduction_indices=1)
            KLD = -0.5 * tf.reduce_mean(1 + logvar_encoder - tf.pow(mu_encoder, 2) - tf.exp(logvar_encoder), reduction_indices=1)
            softmax_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = self.y_, logits = self.y))

            loss = tf.reduce_mean((BCE + KLD + softmax_loss) * lam)

            #loss = tf.clip_by_value(loss, -1e-2, 1e-2)
            #loss = tf.where(tf.is_nan(loss), 1e-2, loss)
            #loss = tf.where(tf.equal(loss, -1e-2), tf.random_normal(loss.shape), loss)
            #loss = tf.where(tf.equal(loss, 1e-2), tf.random_normal(loss.shape), loss)
            
            self.regularized_loss = tf.abs(loss, name = "Regularized_loss")
            correct_prediction = tf.equal(tf.argmax(self.y_, 1), tf.argmax(self.y, 1))
            self.tf_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = "Accuracy")

        with tf.variable_scope("Optimizer"):
            learning_rate=1e-2
            optimizer = tf.train.AdamOptimizer(learning_rate)
            gradients, variables = zip(*optimizer.compute_gradients(self.regularized_loss))
            gradients = [
                None if gradient is None else tf.clip_by_value(gradient, -1, 1)
                for gradient in gradients]
            self.train_op = optimizer.apply_gradients(zip(gradients, variables))
            #self.train_op = optimizer.minimize(self.regularized_loss)
            
        # add op for merging summary
        #self.summary_op = tf.summary.merge_all()
        self.pred = tf.argmax(self.y, axis = 1)
        self.actual = tf.argmax(self.y_, axis = 1)

        # add Saver ops
        self.saver = tf.train.Saver()
        

In [ ]:
import collections

class Train:    
    
    result = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])

    predictions = {}

    results = []
    best_acc = 0
    
    def train(epochs, net, h,f):
        batch_iterations = 200
        os.makedirs("dataset/tf_vae_dense_trained_together_nsl_kdd/hidden layers_{}_features count_{}".format(epochs,h,f),
                    exist_ok = True)
        with tf.Session() as sess:
            #summary_writer_train = tf.summary.FileWriter('./logs/kdd/VAE/training', graph=sess.graph)
            #summary_writer_valid = tf.summary.FileWriter('./logs/kdd/VAE/validation')

            sess.run(tf.global_variables_initializer())
            Train.best_acc = 0
            for epoch in range(1, (epochs+1)):
                x_train, x_valid, y_train, y_valid, = ms.train_test_split(preprocess.x_train, 
                                                                          preprocess.y_train, 
                                                                          test_size=0.2)
                batch_indices = np.array_split(np.arange(x_train.shape[0]), 
                                           batch_iterations)
                                                                          
                for i in batch_indices:
                    _, train_loss = sess.run([net.train_op, 
                                                           net.regularized_loss, 
                                                           ], #net.summary_op
                                                          feed_dict={net.x: x_train[i,:], 
                                                                     net.y_: y_train[i,:], 
                                                                     net.keep_prob:1})
                    
                    #summary_writer_train.add_summary(summary_str, epoch)
                    if((train_loss > 1e4 or np.isnan(train_loss)) and epoch > 1):
                        print("Step {} | High Training Loss: {:.6f} ... Restoring Net".format(epoch, train_loss))
                        net.saver.restore(sess, 
                                          tf.train.latest_checkpoint('dataset/tf_vae_dense_trained_together_nsl_kdd/hidden layers_{}_features count_{}'
                                                                     .format(epochs,h,f)))
                    
                valid_loss, valid_accuracy = sess.run([net.regularized_loss, net.tf_accuracy], #net.summary_op
                                                          feed_dict={net.x: x_valid, 
                                                                     net.y_: y_valid, 
                                                                     net.keep_prob:1})
                
                test_accuracy, test_loss, pred_value, actual_value, y_pred = sess.run([net.tf_accuracy, net.regularized_loss, net.pred, 
                                                                                  net.actual, net.y], #net.summary_op 
                                                                                  feed_dict={net.x: preprocess.x_test, 
                                                                                 net.y_: preprocess.y_test, 
                                                                                 net.keep_prob:1})
                #summary_writer_valid.add_summary(summary_str, epoch)

                if epoch % 1 == 0:
                    print("Step {} | Training Loss: {:.6f} | Test Loss: {:.6f} | Test Accuracy: {:.6f}"
                          .format(epoch, train_loss, test_loss, test_accuracy))

                

                if test_accuracy > Train.best_acc:
                    Train.best_acc = test_accuracy
                    Train.pred_value = pred_value
                    Train.actual_value = actual_value
                    Train.best_parameters = "Hidden Layers:{}, Features Count:{}".format(h, f)
                    
                    net.saver.save(sess, 
                                   "dataset/tf_vae_dense_trained_together_nsl_kdd/hidden layers_{}_features count_{}/model"
                                   .format(epochs,h,f), 
                                   global_step = epoch, 
                                   write_meta_graph=False)
                    
                    curr_pred = pd.DataFrame({"Attack_prob":y_pred[:,-2], "Normal_prob":y_pred[:, -1]})
                    Train.predictions.update({"{}_{}_{}".format(epochs,f,h):
                                              (curr_pred, 
                                               Train.result(epochs, f, h,valid_accuracy, test_accuracy))})
                    #Train.results.append(Train.result(epochs, f, h,valid_accuracy, test_accuracy))
            print("Best Accuracy on Test data: {}".format(Train.best_acc))
            

In [ ]:
import itertools
class Hyperparameters:
#    features_arr = [2, 4, 8, 16, 32, 64, 128, 256]
#    hidden_layers_arr = [2, 4, 6, 10]
    features_arr = [4, 8, 16, 32]
    hidden_layers_arr = [2, 4, 6]

    epochs = [100]
    
    for e, h, f in itertools.product(epochs, hidden_layers_arr, features_arr):
        print("Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
        n = network(2,h,f)
        n.build_layers()
        Train.train(e, n, h,f)
        

Current Layer Attributes - epochs:100 hidden layers:2 features count:4
Step 1 | Training Loss: 0.000111 | Test Loss: 0.000200 | Test Accuracy: 0.801455
Step 2 | Training Loss: 0.000013 | Test Loss: 0.000279 | Test Accuracy: 0.764328
Step 3 | Training Loss: 0.000022 | Test Loss: 0.000284 | Test Accuracy: 0.803318
Step 4 | Training Loss: 0.000067 | Test Loss: 0.000193 | Test Accuracy: 0.806556
Step 5 | Training Loss: 0.000115 | Test Loss: 0.000086 | Test Accuracy: 0.826384
Step 6 | Training Loss: 0.000032 | Test Loss: 0.000271 | Test Accuracy: 0.809883
Step 7 | Training Loss: 0.000032 | Test Loss: 0.000332 | Test Accuracy: 0.828957
Step 8 | Training Loss: 0.000030 | Test Loss: 0.000136 | Test Accuracy: 0.831929
Step 9 | Training Loss: 0.000063 | Test Loss: 0.000188 | Test Accuracy: 0.831707
Step 10 | Training Loss: 0.000046 | Test Loss: 0.000228 | Test Accuracy: 0.840578
Step 11 | Training Loss: 0.000069 | Test Loss: 0.000270 | Test Accuracy: 0.838272
Step 12 | Training Loss: 0.000081 | 

Step 1 | Training Loss: 0.000123 | Test Loss: 0.000978 | Test Accuracy: 0.838272
Step 2 | Training Loss: 0.000003 | Test Loss: 0.000295 | Test Accuracy: 0.775195
Step 3 | Training Loss: 0.000102 | Test Loss: 0.512876 | Test Accuracy: 0.779587
Step 4 | Training Loss: 0.000004 | Test Loss: 0.000256 | Test Accuracy: 0.791696
Step 5 | Training Loss: 0.000133 | Test Loss: 0.000420 | Test Accuracy: 0.815339
Step 6 | Training Loss: 0.000082 | Test Loss: 0.000357 | Test Accuracy: 0.805536
Step 7 | High Training Loss: 101281.718750 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_dense_trained_together_nsl_kdd/hidden layers_100_features count_2/model-1
Step 7 | Training Loss: 0.000242 | Test Loss: 0.000296 | Test Accuracy: 0.745564
Step 8 | Training Loss: 0.000134 | Test Loss: 0.000043 | Test Accuracy: 0.767433
Step 9 | Training Loss: 0.000031 | Test Loss: 0.000130 | Test Accuracy: 0.775461
Step 10 | High Training Loss: 1494144.250000 ... Restoring Net
INFO:tensorflow:

Step 92 | Training Loss: 0.000062 | Test Loss: 0.000222 | Test Accuracy: 0.860273
Step 93 | Training Loss: 0.000027 | Test Loss: 0.000167 | Test Accuracy: 0.882763
Step 94 | Training Loss: 0.000037 | Test Loss: 0.000180 | Test Accuracy: 0.893630
Step 95 | Training Loss: 0.000030 | Test Loss: 0.000157 | Test Accuracy: 0.889815
Step 96 | Training Loss: 0.000013 | Test Loss: 0.000137 | Test Accuracy: 0.891057
Step 97 | Training Loss: 0.000009 | Test Loss: 0.000177 | Test Accuracy: 0.893630
Step 98 | Training Loss: 0.000051 | Test Loss: 0.000161 | Test Accuracy: 0.893586
Step 99 | Training Loss: 0.000010 | Test Loss: 0.000116 | Test Accuracy: 0.892388
Step 100 | Training Loss: 0.000006 | Test Loss: 0.000125 | Test Accuracy: 0.892521
Best Accuracy on Test data: 0.8951383829116821
Current Layer Attributes - epochs:100 hidden layers:2 features count:16
Step 1 | Training Loss: 0.000218 | Test Loss: 0.071237 | Test Accuracy: 0.789345
Step 2 | Training Loss: 0.000059 | Test Loss: 0.000454 | Test

Step 76 | Training Loss: 0.000019 | Test Loss: 0.000134 | Test Accuracy: 0.888707
Step 77 | Training Loss: 0.000077 | Test Loss: 0.000088 | Test Accuracy: 0.894473
Step 78 | Training Loss: 0.000021 | Test Loss: 0.000132 | Test Accuracy: 0.895316
Step 79 | Training Loss: 0.000024 | Test Loss: 0.000115 | Test Accuracy: 0.892965
Step 80 | Training Loss: 0.000026 | Test Loss: 0.000110 | Test Accuracy: 0.887509
Step 81 | Training Loss: 0.000033 | Test Loss: 0.000067 | Test Accuracy: 0.881476
Step 82 | Training Loss: 0.000071 | Test Loss: 0.000159 | Test Accuracy: 0.888751
Step 83 | Training Loss: 0.000052 | Test Loss: 0.000114 | Test Accuracy: 0.878282
Step 84 | Training Loss: 0.000003 | Test Loss: 0.000168 | Test Accuracy: 0.877262
Step 85 | Training Loss: 0.000001 | Test Loss: 0.000123 | Test Accuracy: 0.876597
Step 86 | Training Loss: 0.000028 | Test Loss: 0.000129 | Test Accuracy: 0.871230
Step 87 | Training Loss: 0.000061 | Test Loss: 0.000136 | Test Accuracy: 0.867947
Step 88 | Traini

Step 36 | Training Loss: 0.000090 | Test Loss: inf | Test Accuracy: 0.862181
Step 37 | High Training Loss: inf ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_dense_trained_together_nsl_kdd/hidden layers_100_features count_2/model-33
Step 37 | Training Loss: 0.000023 | Test Loss: inf | Test Accuracy: 0.895050
Step 38 | High Training Loss: inf ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_dense_trained_together_nsl_kdd/hidden layers_100_features count_2/model-37
Step 38 | Training Loss: 0.000039 | Test Loss: 20609562558590747773239296.000000 | Test Accuracy: 0.881964
Step 39 | High Training Loss: 191248468419805184.000000 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_dense_trained_together_nsl_kdd/hidden layers_100_features count_2/model-37
Step 39 | Training Loss: 0.000062 | Test Loss: inf | Test Accuracy: 0.892211
Step 40 | High Training Loss: inf ... Restoring Net
INFO:tensorflow:Restoring parameters

...

Step 24 | Training Loss: 0.000038 | Test Loss: 0.000107 | Test Accuracy: 0.875000
Step 25 | Training Loss: 0.000084 | Test Loss: 0.000149 | Test Accuracy: 0.896780
Step 26 | Training Loss: 0.000024 | Test Loss: 0.000115 | Test Accuracy: 0.880412
Step 27 | Training Loss: 0.000017 | Test Loss: 0.000137 | Test Accuracy: 0.885513
Step 28 | Training Loss: 0.000014 | Test Loss: 0.000199 | Test Accuracy: 0.889594
Step 29 | Training Loss: 0.000006 | Test Loss: 0.000168 | Test Accuracy: 0.875222
Step 30 | Training Loss: 0.000045 | Test Loss: 0.000163 | Test Accuracy: 0.869189
Step 31 | Training Loss: 0.000022 | Test Loss: 0.000176 | Test Accuracy: 0.862003
Step 32 | Training Loss: 0.000019 | Test Loss: 0.000224 | Test Accuracy: 0.868213
Step 33 | Training Loss: 0.000034 | Test Loss: 0.000108 | Test Accuracy: 0.885158
Step 34 | Training Loss: 0.000052 | Test Loss: 0.000076 | Test Accuracy: 0.889505
Step 35 | Training Loss: 0.000009 | Test Loss: 0.000100 | Test Accuracy: 0.886844
Step 36 | Traini

Step 18 | High Training Loss: 57523240024670208.000000 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_dense_trained_together_nsl_kdd/hidden layers_100_features count_4/model-5
Step 18 | Training Loss: 0.000079 | Test Loss: 0.000123 | Test Accuracy: 0.800967
Step 19 | Training Loss: 0.000021 | Test Loss: 0.000286 | Test Accuracy: 0.707461
Step 20 | Training Loss: 0.000020 | Test Loss: 0.000308 | Test Accuracy: 0.714115
Step 21 | Training Loss: 0.000060 | Test Loss: 0.000197 | Test Accuracy: 0.715845
Step 22 | High Training Loss: 131661.593750 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_dense_trained_together_nsl_kdd/hidden layers_100_features count_4/model-18
Step 22 | Training Loss: 0.000020 | Test Loss: 0.000237 | Test Accuracy: 0.825586
Step 23 | Training Loss: 0.000036 | Test Loss: 0.000182 | Test Accuracy: 0.841377
Step 24 | Training Loss: 0.000050 | Test Loss: 0.000171 | Test Accuracy: 0.820795
Step 25 | Training Loss: 0.0

Step 11 | Training Loss: 0.000038 | Test Loss: 0.000268 | Test Accuracy: 0.853442
Step 12 | Training Loss: 0.000008 | Test Loss: 0.000231 | Test Accuracy: 0.867992
Step 13 | Training Loss: 0.000015 | Test Loss: 0.000142 | Test Accuracy: 0.847099
Step 14 | Training Loss: 0.000071 | Test Loss: 0.000147 | Test Accuracy: 0.867060
Step 15 | Training Loss: 0.000047 | Test Loss: 0.000131 | Test Accuracy: 0.879081
Step 16 | Training Loss: 0.000043 | Test Loss: 0.000166 | Test Accuracy: 0.882408
Step 17 | Training Loss: 0.000036 | Test Loss: 0.000155 | Test Accuracy: 0.830953
Step 18 | Training Loss: 0.000034 | Test Loss: 0.000141 | Test Accuracy: 0.855216
Step 19 | Training Loss: 0.000009 | Test Loss: 0.000058 | Test Accuracy: 0.852422
Step 20 | Training Loss: 0.000032 | Test Loss: 0.000045 | Test Accuracy: 0.841377
Step 21 | Training Loss: 0.000005 | Test Loss: 0.000174 | Test Accuracy: 0.788591
Step 22 | High Training Loss: inf ... Restoring Net
INFO:tensorflow:Restoring parameters from data

Step 8 | Training Loss: 0.000084 | Test Loss: 0.000281 | Test Accuracy: 0.664878
Step 9 | Training Loss: 0.000069 | Test Loss: 0.000330 | Test Accuracy: 0.635424
Step 10 | Training Loss: 0.000047 | Test Loss: 0.000221 | Test Accuracy: 0.672951
Step 11 | Training Loss: 0.000030 | Test Loss: 0.000256 | Test Accuracy: 0.744322
Step 12 | Training Loss: 0.000015 | Test Loss: 0.000172 | Test Accuracy: 0.703646
Step 13 | Training Loss: 0.000092 | Test Loss: 0.000392 | Test Accuracy: 0.667406
Step 14 | Training Loss: 0.000088 | Test Loss: 0.000269 | Test Accuracy: 0.794668
Step 15 | Training Loss: 0.000027 | Test Loss: 0.001121 | Test Accuracy: 0.764194
Step 16 | Training Loss: 0.000003 | Test Loss: 0.000421 | Test Accuracy: 0.756742
Step 17 | Training Loss: 0.000094 | Test Loss: 0.000330 | Test Accuracy: 0.686879
Step 18 | Training Loss: 0.000178 | Test Loss: 0.000836 | Test Accuracy: 0.556600
Step 19 | Training Loss: 0.000012 | Test Loss: 0.000813 | Test Accuracy: 0.534333
Step 20 | Training

Step 5 | Training Loss: 0.000113 | Test Loss: 0.000172 | Test Accuracy: 0.801765
Step 6 | Training Loss: 0.000131 | Test Loss: 0.000012 | Test Accuracy: 0.798483
Step 7 | Training Loss: 0.000073 | Test Loss: 0.000135 | Test Accuracy: 0.823146
Step 8 | Training Loss: 0.000174 | Test Loss: 0.000457 | Test Accuracy: 0.793825
Step 9 | Training Loss: 0.000120 | Test Loss: 0.000032 | Test Accuracy: 0.868568
Step 10 | High Training Loss: 15594.648438 ... Restoring Net
INFO:tensorflow:Restoring parameters from dataset/tf_vae_dense_trained_together_nsl_kdd/hidden layers_100_features count_6/model-9
Step 10 | Training Loss: nan | Test Loss: nan | Test Accuracy: 0.569242
Step 11 | Training Loss: nan | Test Loss: nan | Test Accuracy: 0.569242
Step 12 | Training Loss: nan | Test Loss: nan | Test Accuracy: 0.569242
Step 13 | Training Loss: nan | Test Loss: nan | Test Accuracy: 0.569242
Step 14 | Training Loss: nan | Test Loss: nan | Test Accuracy: 0.569242
Step 15 | Training Loss: nan | Test Loss: n

In [ ]:
dict1 = {}
dict2 = []
for k, (v1, v2) in Train.predictions.items():
    dict1.update({k: v1})
    dict2.append(v2)

In [ ]:
Train.predictions = dict1
Train.results = dict2

In [ ]:
df_results = pd.DataFrame(Train.results)

In [ ]:
df_results.sort_values(by = 'test_score', ascending = False)

In [ ]:
pd.Panel(Train.predictions).to_pickle("dataset/tf_vae_dense_trained_together_nsl_kdd_predictions.pkl")
df_results.to_pickle("dataset/tf_vae_dense_trained_together_nsl_kdd_scores.pkl")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    np.set_printoptions(precision=4)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j].round(4),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot(actual_value, pred_value):
    from sklearn.metrics import confusion_matrix
    cm_2labels = confusion_matrix(y_pred = pred_value, y_true = actual_value)
    plt.figure(figsize=[6,6])
    plot_confusion_matrix(cm_2labels, preprocess.output_columns_2labels, normalize = False,
                         title = Train.best_parameters)

In [ ]:
plot(actual_value = Train.actual_value, pred_value = Train.pred_value)